<a href="https://colab.research.google.com/github/O-ElAli/DungeonQLearning/blob/main/DungeonQLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Q-Learning
Q-Learning is a model-free reinforcement learning method used to find the best *policy* in a given state in order to achieve the highest long-term reward.

## Intuition:
You are in an unknown maze, and your goal is to reach the exit. At first, you don’t know the best way to get there. So, you have to try different paths and learn which ones are better than others.

Q-Learning helps you discover these “signposts” by allowing you to evaluate the quality or “value” of actions.

## The Game We Want to Learn
We will apply Q-Learning to a simple dungeon-crawling game that works as follows:

### The Dungeon Layout:

* **Size:** A 5x5 grid  
* **Start position:** The player (🧙) starts in the southwest corner of the grid  
* **Goal:** The target (🏁) is in the northeast corner  
* **Fire:** Dangerous areas (🔥) in the dungeon that give negative points  
* **Dragon:** A dragon (🐉) that kills the player if nearby  
* **Treasure:** A treasure (💰) that grants a high reward when found  

### Game Rules:

* The player can move **north**, **south**, **west**, or **east**  
* Each step gives a reward of **“-0.1” points** (a step costs something, so we want a short path to the goal)  
* Contact with **fire** results in **“-10” points**  
* Being **within 1 tile** of the **dragon** results in **“-∞” points** (the player dies)  
* Stepping on the **treasure** grants **“+50” points**  
* Reaching the **goal** gives **“+100” points** and ends the game



In [ ]:
import numpy as np
import random
import time
from tqdm import tqdm
from IPython.display import clear_output, display

# Definition of Constants and Creation of the Dungeon Grid

In this section of the code, we define the various elements of our dungeon and create the grid that represents the dungeon. We also set the rewards and penalties for different events.

We use a 5x5 NumPy array to define the positions of the different elements in the dungeon.


In [ ]:

EMPTY = 0
GOAL = 1
FIRE = 2
DRAGON = 3
TREASURE = 4
PLAYER = 5


#dungeon.shape=(5,5)
#(rows,columns) = dungeon.shape
#dungeon.shape[0] -> # of rows
#dungeon.shape[1] -> # of cols
dungeon = np.array([
    [EMPTY,  EMPTY, EMPTY, EMPTY, GOAL],
    [DRAGON,  FIRE,  FIRE, EMPTY, EMPTY],
    [EMPTY,  EMPTY, TREASURE, FIRE, EMPTY],
    [EMPTY,  EMPTY, EMPTY,  EMPTY, EMPTY],
    [PLAYER, FIRE, EMPTY, EMPTY, EMPTY]
])

GRID_SIZE = 5
ACTIONS = ['N', 'S', 'W', 'E']
ACTION_COUNT = len(ACTIONS)

step_cost = -1
fire_cost = -50
dragon_cost = -np.inf
treasure_reward = 30
goal_reward = 100

In [ ]:
# Size is based on the dungeon size, and a flag treasure collected/not collected
Q_table = np.zeros((GRID_SIZE * GRID_SIZE * 2, ACTION_COUNT))

In [ ]:
symbol_map = {
    EMPTY: '   ',  # Empty space
    GOAL: '🏁 ',
    FIRE: '🔥 ',
    DRAGON: '🐉 ',
    TREASURE: '💰 ',
    PLAYER: '🧙 '
}

#dungeon.shape=(5,5)
#(rows,columns) = dungeon.shape
#dungeon.shape[0] -> # of rows
#dungeon.shape[1] -> # of cols

def print_dungeon(dungeon):

    top_border = "┌───" + "┬───" * (dungeon.shape[1] - 1) + "┐" # --> 5-1 = 4
    middle_border = "├───" + "┼───" * (dungeon.shape[1] - 1) + "┤"
    bottom_border = "└───" + "┴───" * (dungeon.shape[1] - 1) + "┘"

#.join is for tuples
#myTuple = ("John", "Peter", "Vicky")
#x = "#".join(myTuple)
# RESULT x--> John#Peter#Vicky

    print(top_border)
    for i, row in enumerate(dungeon):
        #print("i=",i,"row=",row)
        #i=row number (0,1,2,3,4), row=organization of cells inside the row (empty, empty, empty, empty, goal) also referred in numbers (0,0,0,0,1)
        #the loop is just the different symbols being printed
        row_str = "│" + "│".join(symbol_map[cell] for cell in row) + "│"
        print(row_str)
        if i < dungeon.shape[0] - 1: #if i (row we're at) is less than max number of rows:
            print(middle_border)
    print(bottom_border)

# Print the dungeon
print_dungeon(dungeon)

┌───┬───┬───┬───┬───┐
│   │   │   │   │🏁 │
├───┼───┼───┼───┼───┤
│🐉 │🔥 │🔥 │   │   │
├───┼───┼───┼───┼───┤
│   │   │💰 │🔥 │   │
├───┼───┼───┼───┼───┤
│   │   │   │   │   │
├───┼───┼───┼───┼───┤
│🧙 │🔥 │   │   │   │
└───┴───┴───┴───┴───┘


## How Does a Q-Learning Episode Work?
In one episode, the agent takes several steps to learn how to behave in an environment.

It uses the following parameters:

### Parameter Explanations:
* **$\alpha$ (Learning Rate)**: Determines how much new information overrides the old Q-value. A high value means new information is given more weight.
* **$\gamma$ (Discount Factor)**: Indicates how important future rewards are. A value close to 1 means future rewards are highly considered.
* **$\epsilon$ (Exploration Rate)**: Determines how often the agent chooses random actions (*exploration*) instead of relying on the best known action (*exploitation*). At the beginning, epsilon is high so the agent can better explore the environment.

### Procedure:
- Choose an action from the action space:
    - With probability `epsilon`: Choose a random valid action (*exploration*)
    - Otherwise: Choose the valid action with the highest Q-value in the current state (*exploitation*)

- Execute the chosen action and observe the new state and the reward received

- Update the Q-value for the current state and action using the *Q-Learning formula*:
    - $ Q(s, a) \leftarrow (1 - \alpha) \cdot Q(s, a) + \alpha \cdot \left[ r + \gamma \cdot \max_{a'} Q(s', a') \right] $

        * $Q(s,a)$: Q-value for state $s$ and action $a$  
        * $r$: Reward for taking action $a$ in state $s$  
        * $s'$: New state reached from $s$ by taking action $a$

- Set the new state $s'$ as the current state

- If the goal state is reached or the agent dies (e.g., from the dragon):  
    - End the episode


In [ ]:
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.5  # Exploration
episodes = 50000

In [ ]:
#state is the position of the player, starts at (4,0) or southwest
def get_state_index(state, tcollected):
    x, y = state #new x and why values
    #tcollected=treasure collected, adds 1 if true and 0 if false
    return (x * GRID_SIZE + y) * 2 + int(tcollected)
    #we return the current value of the grid/state in one number
    # Example: starting position (4, 0), GRID_SIZE = 5, no treasure collected
    # Calculation: (4 * 5 + 0) * 2 + 0 = 20 * 2 + 0 = 40

#up and left = -1
#down and right = +1
def get_new_state(state, action):
    x, y = state
    #starting state = (4,0)
    if action == 0:  # North
        new_state = (max(x - 1, 0), y) #new_state=(3,0) max until x-1=-1
    elif action == 1:  # South
        new_state = (min(x + 1, GRID_SIZE - 1), y) #new_state=(4,0)
    elif action == 2:  # West
        new_state = (x, max(y - 1, 0))#new_state=(4,0)
    elif action == 3:  # East
        new_state = (x, min(y + 1, GRID_SIZE - 1))#new_state=(1,1)

    return new_state

In [ ]:
def get_valid_actions(state):
    x, y = state
    valid_actions = []
    #variable containing a list of all actions the player CAN perform
    #starting position (4,0) --> valid_actions = [0,3]
    #middle of grid (2,2) --> valid_actions = [0,1,2,3]
    if x > 0: valid_actions.append(0)  # North
    if x < GRID_SIZE - 1: valid_actions.append(1)  # South
    if y > 0: valid_actions.append(2)  # West
    if y < GRID_SIZE - 1: valid_actions.append(3)  # East
    return valid_actions

In [ ]:
def get_cost(new_state):
              #     x             y
    if dungeon[new_state[0], new_state[1]] == GOAL:
        return goal_reward

#python syntax, but the loop runs first then checks if the player has any dragons around him in a 1 block radius
    if any(dungeon[new_state[0] + dx, new_state[1] + dy] == DRAGON
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)] #each tuple gets unpacked into dx and dy: (-1,0) -> dx=-1, dy=0
            if 0 <= new_state[0] + dx < GRID_SIZE
            and 0 <= new_state[1] + dy < GRID_SIZE):
        return dragon_cost

    #if player is on a block that has a chest that wasnt collected add a reward
    if dungeon[new_state[0], new_state[1]] == TREASURE and not tcollected:
        return treasure_reward

    if dungeon[new_state[0], new_state[1]] == FIRE:
        return fire_cost

    return 0

In [ ]:
def choose_action(state, tcollected, valid_actions, Q_table, epsilon):
    state_index = get_state_index(state, tcollected)
    #random float between 0 and 1
    #if more than epsilon, take a random decision for the available actions
    if random.uniform(0, 1) < epsilon:
        return random.choice(valid_actions)  # Explore: select a random valid action
    else:
        # Exploit: select the best action based on the Q-table
        #sending the state_index to show where the player is
        #qtable shows what how "valuable" each subsequent option of the player is
        #state(4,0) -> state_index=40
        #qtable[40] = [0.5,-1.0,0.0,0.2] representing [North, South, West, East]
        #np.argmax returns the index (action) with the highest value → 0 in this case (North)
        return np.argmax(Q_table[state_index])

In [ ]:
def update_Q_table(state, new_state, tcollected):
        state_index = get_state_index(state, tcollected) #reminder: tcollected is a boolean that adds 1 or 0 to the state index
        #example: state(4,0), state_index=40
        new_state_index = get_state_index(new_state, tcollected)
        #example (no treasure): new_state(4,1), new_state_index= 42
        reward = step_cost
        #every step starts with a negative reward

        #               (0      ,       4)
        if dungeon[new_state[0], new_state[1]] == GOAL:
            Q_table[state_index, action] = goal_reward
            #add to the table current location/state_index plus the decision taken to the qtable so future runs know where to go
            #example: state(0,3), action east (0,1), add to qtable 100 points as reward
            return False, tcollected #return false to end the game


        #checks if dragon is in a one block radius. If yes, game over
        if any(dungeon[new_state[0] + dx, new_state[1] + dy] == DRAGON
               for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]
               if 0 <= new_state[0] + dx < GRID_SIZE
               and 0 <= new_state[1] + dy < GRID_SIZE):
            Q_table[state_index, action] = dragon_cost
            return False, tcollected
        #if player current state is a treasure that hasn't been collected, add reward and mark as collected
        if dungeon[new_state[0], new_state[1]] == TREASURE and not tcollected:
            reward += treasure_reward
            tcollected = True  # Mark treasure as collected
            new_state_index = get_state_index(new_state, tcollected)
        #if player current state is a fire, add fire cost
        elif dungeon[new_state[0], new_state[1]] == FIRE:
            reward += fire_cost
        #updating the qtable
        #saving old value first for the equation
        old_value = Q_table[state_index, action] #qtable[40] = [0.5,-1.0,0.0,0.2] --> old value = 0.5
        #saving the new_state_index qtable reward values
        future_value = np.max(Q_table[new_state_index])#qtable[38] = [0.2,0.0,0.0,0.3] --> future value = 0.3


        Q_table[state_index, action] = (1 - alpha) * old_value

        Q_table[state_index, action] += alpha * (reward + gamma * future_value)

        return True, tcollected

In [ ]:
for episode in tqdm(range(episodes)):
    state = (4, 0)  # Startin where the player is initally
    tcollected = False

    run = True
    while run:
        valid_actions = get_valid_actions(state)
        action = choose_action(state, tcollected,
                               valid_actions, Q_table, epsilon)

        new_state = get_new_state(state, action)

        run, tcollected = update_Q_table(state,
                                 new_state, tcollected)

        state = new_state

100%|██████████| 50000/50000 [00:03<00:00, 13030.57it/s]


### Computed Policy
After Q-Learning has been completed and the Q-table is filled with the optimal Q-values, we can extract the best action for each state and use it to create the policy.


In [ ]:
def extract_policy(Q_table, tcollected):
    action_map = {
        0: 'N',  # North
        1: 'S',  # South
        2: 'W',  # West
        3: 'E'   # East
    }
    werte = np.zeros((GRID_SIZE, GRID_SIZE), dtype=float)
    policy = np.zeros((GRID_SIZE, GRID_SIZE), dtype=str)

    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            state_index = get_state_index( (x, y), tcollected)
            best_action = np.argmax(Q_table[state_index])
            policy[x, y] = action_map[best_action]
            werte[x, y] = np.max(Q_table[state_index])+get_cost(state_index)

    return werte, policy

def print_policy(werte, policy, treasure_status):
    np.set_printoptions(formatter={'all':lambda x: "{:6.2f}".format(x)})
    print(f"\nQ-Werte (Treasure {'Collected' if treasure_status else 'Not Collected'}):")
    for row in werte:
        print(row.round(2))

    print(f"\nOptimal Policy (Treasure {'Collected' if treasure_status else 'Not Collected'}):")
    for row in policy:
        print(" ".join(row))

werte_not_collected, policy_not_collected = extract_policy(Q_table, tcollected=False)
werte_collected, policy_collected = extract_policy(Q_table, tcollected=True)

print_policy(werte_not_collected, policy_not_collected, treasure_status=False)
print_policy(werte_collected, policy_collected, treasure_status=True)


TypeError: 'int' object is not subscriptable

### Executing the Policy
The following code visualizes the policy and guides our player to the goal.


In [ ]:
def animate_step(dungeon, player_pos, tcollected):
    display_dungeon = np.copy(dungeon)

    px, py = player_pos

    if tcollected and display_dungeon[px, py] == TREASURE:
        display_dungeon[px, py] = EMPTY

    display_dungeon[px, py] = PLAYER

    top_border = "┌───" + "┬───" * (dungeon.shape[1] - 1) + "┐"
    middle_border = "├───" + "┼───" * (dungeon.shape[1] - 1) + "┤"
    bottom_border = "└───" + "┴───" * (dungeon.shape[1] - 1) + "┘"

    print_grid = [top_border]

    for x in range(GRID_SIZE):
        row = '│'
        for y in range(GRID_SIZE):
            if tcollected and display_dungeon[x, y] == TREASURE:
                row += symbol_map[EMPTY] + '│'  # Treasure is hidden once collected
            else:
                row += symbol_map[display_dungeon[x, y]] + '│'
        print_grid.append(row)
        if x < GRID_SIZE - 1:
            print_grid.append(middle_border)

    print_grid.append(bottom_border)

    clear_output(wait=True)

    for line in print_grid:
        print(line)

In [ ]:
import time
from IPython.display import clear_output

def animate_policy(Q_table, dungeon, start_pos=(4, 0), sleep_time=2):
    werte_not_collected, policy_not_collected = extract_policy(Q_table, tcollected=False)
    werte_collected, policy_collected = extract_policy(Q_table, tcollected=True)

    player_pos = start_pos
    tcollected = False
    accumulated_reward = 0

    px, py = player_pos
    dungeon[px, py] = EMPTY  # Start by clearing the player’s initial position

    while True:
        animate_step(dungeon, player_pos, tcollected)

        state_index = get_state_index(player_pos, tcollected)
        best_action = np.argmax(Q_table[state_index])

        new_pos = get_new_state(player_pos, best_action)
        x, y = new_pos

        if dungeon[x, y] == TREASURE:
            tcollected = True
            dungeon[x, y] = EMPTY  # Mark treasure as collected

        if dungeon[x, y] == GOAL:
            clear_output(wait=True)
            animate_step(dungeon, new_pos, tcollected)
            print("Reached goal!!")
            break

        if dungeon[x, y] == DRAGON:
            clear_output(wait=True)
            animate_step(dungeon, new_pos, tcollected)
            print("Died to the Dragon!!")
            break

        player_pos = new_pos

        time.sleep(sleep_time)

copied_dungeon = np.copy(dungeon)

In [ ]:
animate_policy(Q_table, copied_dungeon)

┌───┬───┬───┬───┬───┐
│   │   │   │   │🧙 │
├───┼───┼───┼───┼───┤
│🐉 │🔥 │🔥 │   │   │
├───┼───┼───┼───┼───┤
│   │   │   │🔥 │   │
├───┼───┼───┼───┼───┤
│   │   │   │   │   │
├───┼───┼───┼───┼───┤
│   │🔥 │   │   │   │
└───┴───┴───┴───┴───┘
Reached goal!!
